In [ ]:
from pydantic_settings import BaseSettings, SettingsConfigDict

from vocode.helpers import create_turn_based_microphone_input_and_speaker_output
from vocode.turn_based.agent.chat_gpt_agent import ChatGPTAgent


: 